In [19]:
# Folder: scripts/ingest_compustat.py
import wrds
import pandas as pd
from sqlalchemy import create_engine

In [21]:
# WRDS login and parameters
start_date = '2005-01-01'
end_date = '2023-12-31'
db_path = "/Users/niveditavats/Documents/Projects_CC/equity_analytics_pipelines/database/investment_data.sqlite"

In [20]:
# start_date = "2005-01-01"
# end_date = "2023-12-31"
# db_path = "/Users/niveditavats/Documents/Projects_CC/equity_analytics_pipelines/database/investment_data.sqlite"


In [23]:
# Connect to WRDS and SQLite
conn = wrds.Connection()
engine = create_engine(f"sqlite:///{db_path}")

Enter your WRDS username [niveditavats]: niveditavats
Enter your password: ········


OperationalError: (psycopg2.OperationalError) connection to server at "wrds-pgdata.wharton.upenn.edu" (165.123.60.118), port 9737 failed: FATAL:  PAM authentication failed for user "niveditavats"

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [12]:
# Simulated Compustat data
data = pd.DataFrame({
    'gvkey': ['001000', '001000'],
    'datadate': ['2022-09-30', '2022-12-31'],
    'seq': [1000, 1100], 'ceq': [500, 550], 'at': [3000, 3200],
    'lt': [2000, 2100], 'txditc': [0, 0], 'txdb': [0, 0], 'itcb': [0, 0],
    'pstkrv': [0, 0], 'pstkl': [0, 0], 'pstk': [0, 0],
    'capx': [100, 120], 'oancf': [300, 310],
    'sale': [1500, 1600], 'cogs': [800, 850], 'xint': [50, 55], 'xsga': [100, 105]
})

In [13]:
# Add dummy symbol for merging
data['symbol'] = 'AAPL'

In [14]:

# Calculate derived metrics
data['datadate'] = pd.to_datetime(data['datadate'])
data['book_equity'] = data['seq'].fillna(data['ceq'])
data['profitability'] = (data['sale'] - data['cogs'] - data['xsga'] - data['xint']) / data['at']



In [15]:

# Store to SQLite
data.to_sql("compustat_quarterly", con=engine, if_exists="replace", index=False)
print("Compustat quarterly data ingested and stored in database.")


Compustat quarterly data ingested and stored in database.


In [16]:
# Folder: scripts/ingest_crsp.py
import yfinance as yf
import pandas as pd
from sqlalchemy import create_engine

In [18]:
symbols = ["AAPL"]  # Limit to 1 symbol to avoid rate limiting
start_date = "2020-01-01"
end_date = "2023-12-31"
db_path = "../database/investment_data.sqlite"

engine = create_engine(f"sqlite:///{db_path}")

try:
    prices = yf.download(tickers=symbols, start=start_date, end=end_date, progress=False, group_by='ticker')
except Exception as e:
    print(f"Error during download: {e}")
    prices = pd.DataFrame()

if not prices.empty and 'AAPL' in prices.columns.levels[0]:
    prices = prices['AAPL'].reset_index()
    prices = prices.rename(columns={'Adj Close': 'adjusted', 'Date': 'date'})
    prices['symbol'] = 'AAPL'
    prices['date'] = pd.to_datetime(prices['date'])
    prices['ret'] = prices['adjusted'].pct_change()
    prices.dropna(subset=['ret'], inplace=True)
    prices[['symbol', 'date', 'ret']].to_sql("crsp_daily", con=engine, if_exists="replace", index=False)
    print("CRSP daily data ingested and stored in database.")
elif prices.empty:
    print("Download failed or returned no data. You may have been rate-limited. Try again in a few minutes.")
else:
    print("Unexpected data format. 'AAPL' not found in downloaded columns.")




1 Failed download:
['AAPL']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


Download failed or returned no data. You may have been rate-limited. Try again in a few minutes.


In [10]:

engine = create_engine(f"sqlite:///{db_path}")


In [11]:

prices = yf.download(tickers=symbols, start=start_date, end=end_date, progress=False)
prices = prices.stack(level=1).reset_index().rename(columns={
    'level_1': 'symbol', 'Adj Close': 'adjusted', 'Date': 'date'
})
prices['date'] = pd.to_datetime(prices['date'])
prices['ret'] = prices.groupby('symbol')['adjusted'].pct_change()
prices.dropna(subset=['ret'], inplace=True)
prices[['symbol', 'date', 'ret']].to_sql("crsp_daily", con=engine, if_exists="replace", index=False)
print("CRSP daily data ingested and stored in database.")


YF.download() has changed argument auto_adjust default to True



3 Failed downloads:
['MSFT', 'AAPL', 'IBM']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


KeyError: 'symbol'